In [16]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas_profiling
import statistics
import copy

In [18]:
traindf = pd.read_csv("train.csv")
testdf = pd.read_csv("test.csv")
ss = pd.read_csv("ss.csv")

In [4]:
traindf.profile_report()

In [19]:
# Changing the building painted values into numeric values for both test and train data
# Test dataset
test_bp = testdf['Building_Painted']
test_bp= test_bp.map({'V': 0, 'N': 1})
testdf['Building_Painted'] = test_bp

# Train dataset
train_bp = traindf['Building_Painted']
train_bp= train_bp.map({'V': 0, 'N': 1})
traindf['Building_Painted'] = train_bp



# Changing the building fenced values into numeric values for both test and train data
# Test dataset
test_bf = testdf['Building_Fenced']
test_bf= test_bf.map({'V': 0, 'N': 1})
testdf['Building_Fenced'] = test_bf

# Train dataset
train_bf = traindf['Building_Fenced']
train_bf= train_bf.map({'V': 0, 'N': 1})
traindf['Building_Fenced'] = train_bf


# print(train_bf, test_bf)


In [20]:
# Changing the garden values into numeric values for both test and train data
# Test dataset
test_ga = testdf['Garden']
test_ga= test_ga.map({'V': 1, 'O': 0})
testdf['Garden'] = test_ga

# Train dataset
train_ga = traindf['Garden']
train_ga= train_ga.map({'V': 1, 'O': 0})
traindf['Garden'] = train_ga

# Changing the settlement values into numeric values for both test and train data
# Test dataset
test_se = testdf['Settlement']
test_se= test_se.map({'R': 0, 'U': 1})
testdf['Settlement'] = test_se

# Train dataset
train_se = traindf['Settlement']
train_se= train_se.map({'R': 0, 'U': 1})
traindf['Settlement'] = train_se


# print(train_se, test_se)


In [24]:
bd = list(testdf["Building Dimension"])
tsh = sorted(bd)[int((1/10)*len(bd))]
tsh2 = sorted(bd)[int((9/10)*len(bd))]
# 10th and 90th percentile of building size
logre = copy.deepcopy(traindf)
# To construct our pseudologistic regression formula, we use copy.deepcopy to create a clone
# of the traindf so we can change it without changing the original.
logre1 = logre.loc[logre['Building Dimension'] > tsh]
logre2 = logre.loc[logre['Building Dimension'] < tsh2]
# Creating dataframes based on the top and bottom 10%

In [25]:
low10 = statistics.mean(list(logre2["Claim"]))
high90 = statistics.mean(list(logre1["Claim"]))
# Portion of top and bottom segments that have claims filed

In [26]:
# Creating the linear formula based on the slope between two points
# and the formula y = mx + b
tsh3 = sorted(bd)[int((1/20)*len(bd))]
tsh4 = sorted(bd)[int((19/20)*len(bd))]
m = (high90-low10)/(tsh4-tsh3)
b = m*tsh3-low10

In [27]:
# For each of the next five features, we prepare the Naive Bayes by making a list
# with the probability of a claim given a specific feature.
bpt = list(testdf["Building_Painted"])
bptprobarray = [statistics.mean(traindf.loc[traindf["Building_Painted"] == i]["Claim"]) for i in range(2)]
bptprobs = [bptprobarray[bpt[i]] for i in range(len(bpt))]

In [28]:
bft = list(testdf["Building_Fenced"])
bftprobarray = [statistics.mean(traindf.loc[traindf["Building_Fenced"] == i]["Claim"]) for i in range(2)]
bftprobs = [bftprobarray[int(bft[i])] for i in range(len(testdf))]

In [30]:
stl = list(testdf["Settlement"])
stlprobarray = [statistics.mean(traindf.loc[traindf["Settlement"] == i]["Claim"]) for i in range(2)]
stlprobs = [stlprobarray[stl[i]] for i in range(len(testdf))]

In [31]:
rd = list(testdf["Residential"])
rdprobarray = [statistics.mean(traindf.loc[traindf["Residential"] == i]["Claim"]) for i in range(2)]
rdprobs = [rdprobarray[rd[i]] for i in range(len(testdf))]

In [32]:
bt = list(testdf["Building_Type"])
btprobarray = [statistics.mean(traindf.loc[traindf["Building_Type"] == i]["Claim"]) for i in range(1, 5)]
btprobs = [btprobarray[bt[i]-1] for i in range(len(testdf))]

In [33]:
# We choose which features to include based on the correlogram. We don't
# want to include two features with a 0.999 correlation. That would be redundant.
traindf.corr()

,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,Claim
YearOfObservation,1.000000,0.025434,-0.040391,-0.056881,0.120373,-0.120704,-0.120103,0.009128,0.005490,-0.029492,-0.016367
Insured_Period,0.025434,1.000000,-0.037620,0.037077,0.047091,-0.046858,-0.046504,0.007473,-0.017811,-0.003445,0.088715
Residential,-0.040391,-0.037620,1.000000,0.032927,0.082448,-0.082674,-0.082684,0.057481,0.334039,-0.042058,0.063509
Building_Painted,-0.056881,0.037077,0.032927,1.000000,-0.574757,0.574731,0.575081,-0.092501,0.048541,0.096810,-0.029857
Building_Fenced,0.120373,0.047091,0.082448,-0.574757,1.000000,-0.999720,-0.999441,0.132367,0.020430,-0.171201,0.051657
Garden,-0.120704,-0.046858,-0.082674,0.574731,-0.999720,1.000000,0.999720,-0.131870,-0.021330,0.171456,-0.052093
Settlement,-0.120103,-0.046504,-0.082684,0.575081,-0.999441,0.999720,1.000000,-0.132320,-0.020320,0.171117,-0.052019
Building Dimension,0.009128,0.007473,0.057481,-0.092501,0.132367,-0.131870,-0.132320,1.000000,0.007935,0.207624,0.297089
Building_Type,0.005490,-0.017811,0.334039,0.048541,0.020430,-0.021330,-0.020320,0.007935,1.000000,-0.146608,0.112168
Date_of_Occupancy,-0.029492,-0.003445,-0.042058,0.096810,-0.171201,0.171456,0.171117,0.207624,-0.146608,1.000000,0.019170


In [34]:
dimen_pred = [m*bd[i]+b if m*bd[i]+b < 0.55 else 0.55 for i in range(len(testdf))]  # finally, the pseudologistic regression line
probabilities = [dimen_pred[i]*btprobs[i]*bptprobs[i]*stlprobs[i]*rdprobs[i] for i in range(len(btprobs))]  # putting it all together

In [35]:
threshold = sorted(probabilities)[int((5360/7160)*len(probabilities))]
# Predicting 1 for all buildings more likely than the threshold.
predictions = [1 if probabilities[i] > threshold else 0 for i in range(len(probabilities))]

In [36]:
print(statistics.mean(predictions))
# Just to check what percentage of the values I'm predicting

0.2512218963831867


In [37]:
# Writing the csv that we'll submit
ss1 = pd.DataFrame(list(zip(list(testdf["Customer Id"]), predictions)), columns =["Customer Id", "Claim"])
#ss1["Claim"] = predictions
#ss1["Customer ID"] = list(testdf["Customer ID"])
ss1.to_csv('ss1.csv', index=False)